In [1]:
import pandas as pd
df = pd.read_csv("../TextFiles/hotel-review-train.csv")

In [11]:
def get_w2vdf(df):
    w2v_df = pd.DataFrame(df["Description"]).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = w2v_df[i][0].split(" ")
#         print(w2v_df[i]) #['The', 'room', 'was', 'kind',....]
    return w2v_df

In [17]:
#Preprocessing 
from nltk.corpus import stopwords
import re

def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text

def change_lower(text):
    text = text.lower()
    return text

stopwords_list = stopwords.words("english")
def remover(text):
    text_tokens = text.split(" ")
    final_list = [word for word in text_tokens if not word in stopwords_list]
    text = ' '.join(final_list)
    return text

df[["Description"]] = df[["Description"]].astype(str)
df["Description"] = df["Description"].apply(change_lower)
df["Description"] = df["Description"].apply(clean_data)
df["Description"] = df["Description"].apply(remover)

In [18]:
w2v_df = get_w2vdf(df)

In [22]:
from gensim.models import Word2Vec
import multiprocessing

def train_w2v(w2v_df):
    """
    min_count: The minimum number of occurrences of 
    the target word in the corpus. 
    large corpus high limit increases success
    keep it small for small datasets.
    
    window: It is the number of neighboring words that 
    will directly affect the vector calculations of 
    the target expression
    
    size : It is the size of the vector to be created 
    for each element.
    
    alpha : Initial learning rate
    
    min_alpha : It is the minimum value at which the 
    learning rate will decrease linearly during training.
    
    sg : Specifies how the training algorithm will work. 
    If value of sg is 1, skip-gram algorithm is used, 
    otherwise the CBOW algorithm is used.
    """
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=4,
                         window=4,
                         vector_size=300, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         sg = 1,
                         workers=cores-1)
    w2v_model.build_vocab(w2v_df, progress_per=10000)
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model
    
w2v_model = train_w2v(w2v_df)

In [27]:
w2v_model.wv.get_vector("great")

array([-9.61987209e-03,  1.02379218e-01,  1.59424603e-01,  4.93593514e-02,
       -7.78454766e-02,  1.00491092e-01,  2.31843237e-02,  2.81850785e-01,
       -5.16340137e-02, -3.69267724e-02,  4.30848822e-02, -2.58787572e-01,
       -7.56992176e-02, -1.39572993e-01, -1.42266169e-01, -1.61861882e-01,
        8.89398530e-02,  9.01437737e-03,  1.61843255e-01,  8.66483301e-02,
       -1.93523765e-01,  1.32635355e-01, -8.28197226e-02,  3.50892469e-02,
        1.78847998e-01,  1.17240220e-01, -1.55176535e-01, -6.86076358e-02,
        3.61523271e-04,  6.40983693e-03, -1.06722713e-02,  5.47895469e-02,
        4.03645374e-02,  1.53935671e-01,  1.68180920e-03, -1.03976250e-01,
       -3.54887396e-02, -1.13739111e-01, -3.15207131e-02,  8.18280429e-02,
       -2.44706914e-01, -4.39523198e-02,  9.72939506e-02, -7.56599102e-03,
        1.12471908e-01,  1.02618553e-01, -7.78855383e-02, -4.94308844e-02,
       -2.96135508e-02,  1.24883719e-01,  3.18891630e-02, -6.82802126e-02,
       -5.31772617e-03,  

In [28]:
len(w2v_model.wv.get_vector("great"))

300

In [29]:
w2v_model.wv.most_similar(positive=["great"])

[('good', 0.7016151547431946),
 ('excellent', 0.6685694456100464),
 ('wonderful', 0.6236960291862488),
 ('fantastic', 0.6147310137748718),
 ('nice', 0.5785568952560425),
 ('amazing', 0.5357890129089355),
 ('perfect', 0.5231554508209229),
 ('awesome', 0.5192371606826782),
 ('fabulous', 0.48088306188583374),
 ('terrific', 0.4674989879131317)]

In [31]:
w2v_model.wv.most_similar(positive=["husband"])

[('wife', 0.6497529745101929),
 ('boyfriend', 0.46188926696777344),
 ('girlfriend', 0.4330250024795532),
 ('son', 0.42851123213768005),
 ('daugther', 0.4112330973148346),
 ('daughter', 0.4013725519180298),
 ('partner', 0.39843371510505676),
 ('mother', 0.39489102363586426),
 ('spouse', 0.36922213435173035),
 ('friend', 0.3613705039024353)]

In [32]:
w2v_model.wv.most_similar(positive=["hotel"])

[('property', 0.5878936648368835),
 ('hotels', 0.47673025727272034),
 ('place', 0.46287646889686584),
 ('location', 0.4345657527446747),
 ('motel', 0.4076734781265259),
 ('stay', 0.38551947474479675),
 ('facility', 0.3808191120624542),
 ('halcyon', 0.36929813027381897),
 ('academic', 0.36583253741264343),
 ('griffon', 0.36425426602363586)]

In [33]:
w2v_model.wv.most_similar(positive=["boston"])

[('seattle', 0.47903019189834595),
 ('dc', 0.4526003301143646),
 ('copley', 0.4485768675804138),
 ('quincy', 0.4480293393135071),
 ('newbury', 0.42844507098197937),
 ('fenway', 0.4228980839252472),
 ('chicago', 0.4223571717739105),
 ('commons', 0.4186369478702545),
 ('philly', 0.4021013081073761),
 ('dallas', 0.3848699629306793)]

In [34]:
w2v_model.wv.most_similar(positive=["valet"])

[('parking', 0.6088953018188477),
 ('garage', 0.46016934514045715),
 ('car', 0.43681827187538147),
 ('self', 0.4172986149787903),
 ('parked', 0.3999088704586029),
 ('priviledges', 0.3933953642845154),
 ('privledges', 0.3904281556606293),
 ('valets', 0.388712614774704),
 ('inout', 0.37371209263801575),
 ('parking\noverall', 0.37307727336883545)]

In [35]:
w2v_model.wv.most_similar(positive=["road"])

[('street', 0.48912009596824646),
 ('highway', 0.4085976779460907),
 ('freeway', 0.36973994970321655),
 ('rosecrans', 0.36814284324645996),
 ('expressway', 0.3667968809604645),
 ('sepulveda', 0.3662998378276825),
 ('warrior', 0.3662107288837433),
 ('across', 0.34583133459091187),
 ('trecolori', 0.30878928303718567),
 ('trattoria', 0.30718088150024414)]

In [36]:
w2v_model.wv.most_similar(positive=["customer"])

[('service', 0.471053808927536),
 ('blames', 0.3998851776123047),
 ('salesman', 0.3910908102989197),
 ('competence', 0.37737327814102173),
 ('attitude', 0.3760489821434021),
 ('professionalism', 0.37020155787467957),
 ('hospitality', 0.3675953447818756),
 ('costumer', 0.3628304600715637),
 ('recognizes', 0.35927265882492065),
 ('skills', 0.3562058210372925)]

In [44]:
w2v_model.wv.most_similar(positive=["tery"]) #word that ain't present in the model

KeyError: "Key 'tery' not present in vocabulary"

In [ ]:
w2v_model.wv.